<a href="https://colab.research.google.com/github/Gangasri17/IT-Operations-Management-System-/blob/main/IT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas

In [ ]:
pip install scikit-learn

In [ ]:
pip install requests

In [ ]:
pip install sqlalchemy

In [ ]:
pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 5.4 MB/s eta 0:00:00


In [ ]:
pip install paramiko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.1 MB/s eta 0:00:00


In [ ]:
pip install tensorflow

In [ ]:
pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.3 MB/s eta 0:00:00


In [ ]:
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.4 MB/s eta 0:00:00


In [ ]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 11.9 MB/s eta 0:00:00


In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
from faker import Faker
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# Initialize Faker generator
fake = Faker()

# Number of assets to generate
num_assets = 1000
num_data_points = 1000

# --- Generate synthetic data for IT assets ---
data_assets = {
    'asset_id': [fake.uuid4() for _ in range(num_assets)],
    'asset_type': [fake.random_element(elements=('server', 'laptop', 'network_switch')) for _ in range(num_assets)],
    'age_years': [fake.random_int(min=1, max=10) for _ in range(num_assets)],
    'usage_hours_per_week': [fake.random_int(min=10, max=80) for _ in range(num_assets)],
    'maintenance_cost_last_year': [fake.random_int(min=0, max=500) for _ in range(num_assets)],
    'remaining_lifespan_years': [fake.random_int(min=0, max=5) for _ in range(num_assets)]
}

df_assets = pd.DataFrame(data_assets)

# Adding additional statistics
df_assets['maintenance_needed'] = df_assets['remaining_lifespan_years'].apply(lambda x: 1 if x < 2 else 0)
asset_summary = df_assets.groupby('asset_type').agg(
    total_assets=('asset_id', 'count'),
    average_age=('age_years', 'mean'),
    average_maintenance_cost=('maintenance_cost_last_year', 'mean')
).reset_index()

# --- Generate synthetic data for network metrics ---
network_data = {
    'timestamp': [fake.date_time_this_decade() for _ in range(num_data_points)],
    'network_traffic_mb': np.random.normal(100, 20, num_data_points),
    'cpu_utilization_percent': np.random.normal(60, 10, num_data_points),
    'memory_utilization_percent': np.random.normal(70, 15, num_data_points)
}

df_network = pd.DataFrame(network_data)

# --- Automated IT Asset Management ---
features_assets = ['age_years', 'usage_hours_per_week', 'maintenance_cost_last_year']
target_assets = 'maintenance_needed'

X_assets = df_assets[features_assets]
y_assets = df_assets[target_assets]

X_train_assets, X_test_assets, y_train_assets, y_test_assets = train_test_split(X_assets, y_assets, test_size=0.2, random_state=42)

model_assets = RandomForestClassifier()
model_assets.fit(X_train_assets, y_train_assets)
y_pred_assets = model_assets.predict(X_test_assets)

accuracy_assets = accuracy_score(y_test_assets, y_pred_assets)

# --- AI-Powered Network Monitoring ---
model_network = IsolationForest(contamination=0.05)  # Adjust contamination based on anomaly detection threshold
model_network.fit(df_network[['network_traffic_mb', 'cpu_utilization_percent', 'memory_utilization_percent']])

# Predict anomalies
df_network['anomaly'] = model_network.predict(df_network[['network_traffic_mb', 'cpu_utilization_percent', 'memory_utilization_percent']])

# --- Comprehensive System Dashboard ---
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("IT Operations Dashboard"),

    html.Div([
        html.H2("Asset Management"),
        html.P(f"Total Assets: {num_assets}"),
        html.P(f"Asset Types: {', '.join(df_assets['asset_type'].unique())}"),
        html.P(f"Average Age: {df_assets['age_years'].mean():.1f} years"),
        html.Div([
            dcc.Graph(
                id='asset-type-graph',
                figure={
                    'data': [
                        go.Bar(x=asset_summary['asset_type'], y=asset_summary['total_assets'], name='Total Assets'),
                        go.Bar(x=asset_summary['asset_type'], y=asset_summary['average_age'], name='Average Age'),
                        go.Bar(x=asset_summary['asset_type'], y=asset_summary['average_maintenance_cost'], name='Avg Maintenance Cost'),
                    ],
                    'layout': go.Layout(
                        title='Asset Summary by Type',
                        barmode='group'
                    )
                }
            )
        ])
    ], style={'width': '30%', 'display': 'inline-block', 'vertical-align': 'top'}),

    html.Div([
        html.H2("Network Monitoring"),
        dcc.Graph(
            id='network-traffic-graph',
            figure={
                'data': [
                    go.Scatter(x=df_network['timestamp'], y=df_network['network_traffic_mb'], mode='lines', name='Network Traffic (MB)'),
                ],
                'layout': go.Layout(
                    title='Network Traffic Over Time'
                )
            }
        ),
        dcc.Graph(
            id='cpu-utilization-graph',
            figure={
                'data': [
                    go.Scatter(x=df_network['timestamp'], y=df_network['cpu_utilization_percent'], mode='lines', name='CPU Utilization (%)'),
                ],
                'layout': go.Layout(
                    title='CPU Utilization Over Time'
                )
            }
        ),
        dcc.Graph(
            id='memory-utilization-graph',
            figure={
                'data': [
                    go.Scatter(x=df_network['timestamp'], y=df_network['memory_utilization_percent'], mode='lines', name='Memory Utilization (%)'),
                ],
                'layout': go.Layout(
                    title='Memory Utilization Over Time'
                )
            }
        ),
        dcc.Graph(
            id='anomaly-graph',
            figure={
                'data': [
                    go.Scatter(
                        x=df_network[df_network['anomaly'] == -1]['timestamp'],
                        y=df_network[df_network['anomaly'] == -1]['network_traffic_mb'],
                        mode='markers',
                        marker=dict(color='red', size=10),
                        name='Anomalies'
                    )
                ],
                'layout': go.Layout(
                    title='Network Traffic Anomalies',
                    xaxis=dict(type='date')
                )
            }
        )
    ], style={'width': '65%', 'display': 'inline-block', 'vertical-align': 'top'}),

    html.Div([
        html.H2("Data Upload"),
        dcc.Upload(
            id='upload-data',
            children=html.Div(['Drag and Drop or ', html.A('Select Files')]),
            style={
                'width': '100%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            }
        ),
        html.Div(id='output-data-upload')
    ], style={'width': '100%', 'display': 'inline-block', 'vertical-align': 'top'})
])

# Callback to handle file upload
@app.callback(Output('output-data-upload', 'children'),
              [Input('upload-data', 'contents')],
              [dash.dependencies.State('upload-data', 'filename')])
def update_output(contents, filename):
    if contents is not None:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        try:
            df_uploaded = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
            return html.Div([
                html.H5(filename),
                dcc.Graph(
                    id='uploaded-data-graph',
                    figure={
                        'data': [
                            go.Scatter(x=df_uploaded['timestamp'], y=df_uploaded['value'], mode='lines', name='Value'),
                        ],
                        'layout': go.Layout(
                            title='Uploaded Data Visualization'
                        )
                    }
                )
            ])
        except Exception as e:
            return html.Div([
                'There was an error processing this file.',
                html.Pre(str(e))
            ])


if __name__ == '__main__':
    app.run_server(debug=True)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names



<IPython.core.display.Javascript object>